In [ ]:
import pandas as pd
df=pd.read_csv("tracks_features.csv")
print(df.head())

<ipython-input-3-b5972f6d57c7>:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("tracks_features.csv")


                       id                   name                      album  \
0  7lmeHLHBe4nmXzuXc0HDjk                Testify  The Battle Of Los Angeles   
1  1wsRitfRRtWyEapl0q22o8        Guerrilla Radio  The Battle Of Los Angeles   
2  1hR0fIFK2qRG3f3RF70pb7       Calm Like a Bomb  The Battle Of Los Angeles   
3  2lbASgTSoDO7MTuLAXlTW0              Mic Check  The Battle Of Los Angeles   
4  1MQTmpYOZ6fcMQc56Hdo7T  Sleep Now In the Fire  The Battle Of Los Angeles   

                 album_id                       artists  \
0  2eia0myWFgoHuttJytCxgX  ['Rage Against The Machine']   
1  2eia0myWFgoHuttJytCxgX  ['Rage Against The Machine']   
2  2eia0myWFgoHuttJytCxgX  ['Rage Against The Machine']   
3  2eia0myWFgoHuttJytCxgX  ['Rage Against The Machine']   
4  2eia0myWFgoHuttJytCxgX  ['Rage Against The Machine']   

                   artist_ids  track_number  disc_number explicit  \
0  ['2d0hyoQ5ynDBnkvAbJKORj']             1            1    False   
1  ['2d0hyoQ5ynDBnkvAbJKORj']   

In [2]:
from sklearn.preprocessing import StandardScaler
# Define the list of numeric features to use for recommendations
features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
            'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'year']

# Create a feature matrix from these columns
X = df[features].values

# Standardize the feature matrix
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(X_scaled)

[[-0.12156168  1.5897166   0.91801558 ...  0.00878068 -0.23862149
  -0.68818274]
 [ 0.55856939  1.51845377  0.86573926 ... -0.45105655 -0.26286779
  -0.68818274]
 [-0.93877343  1.56256886  0.91443501 ...  1.03806488  0.30856944
  -0.68818274]
 ...
 [ 1.53922349  0.97210543  0.83766759 ...  0.14098469  0.84147029
   0.55136696]
 [ 0.90654342  1.17571351  0.71907911 ...  0.23763261  0.46615501
   0.55136696]
 [ 1.28087913  0.67348025  0.36231113 ...  0.0115282   0.34610714
   0.55136696]]


In [4]:
# Build the Annoy index (using 'angular' distance, which is equivalent to cosine similarity)
from annoy import AnnoyIndex
f = X_scaled.shape[1]
annoy_index = AnnoyIndex(f, 'angular')

# Add items to the index (this might take a while for 1.2M items)
for i, vector in enumerate(X_scaled):
    if i % 100000 == 0:
        print(f"Adding item {i}")
    annoy_index.add_item(i, vector.tolist())

# Build the index with a given number of trees (more trees = higher accuracy but slower indexing)
n_trees = 10
print("Building Annoy index...")
annoy_index.build(n_trees)
print("Annoy index built.")





Adding item 0
Adding item 100000
Adding item 200000
Adding item 300000
Adding item 400000
Adding item 500000
Adding item 600000
Adding item 700000
Adding item 800000
Adding item 900000
Adding item 1000000
Adding item 1100000
Adding item 1200000
Building Annoy index...
Annoy index built.


In [13]:
def recommend_song_annoy(song_title, df, X_scaled, annoy_index, top_n=5):
    """Returns a list of recommended song names based on Annoy search."""
    indices = df.index[df['name'] == song_title].tolist()
    if not indices:
        return []  # Return an empty list if the song is not found
    idx = indices[0]

    # Get the top_n + 1 nearest neighbors (including the song itself)
    nearest_indices = annoy_index.get_nns_by_item(idx, top_n + 1)

    # Exclude the query song itself (usually the first one)
    nearest_indices = [i for i in nearest_indices if i != idx][:top_n]

    return df.iloc[nearest_indices]['name'].tolist()  # Extract only song names

In [ ]:
#Single-song Recommendation
query_song = 'Dancing With Your Ghost'
recommended_songs = recommend_song_annoy(query_song, df, X_scaled, annoy_index, top_n=5)
print("\nRecommended Songs:")
print(recommended_songs)


Recommended Songs:
['im used to it', 'Wet Dreams', 'Pharaoh', 'Love Gets You Twisted - Acoustic', "I'll Be Fine Somehow"]


In [ ]:
from collections import defaultdict
#Getting the list of most common songs in list 
def common_songs_priority(*lists):
    song_counts = defaultdict(int)
    song_positions = {}

    # Count occurrences and track first positions
    for i, song_list in enumerate(lists):
        song_set = set(song_list)  # Remove duplicates in a single list
        for song in song_set:
            song_counts[song] += 1
            if song not in song_positions:
                song_positions[song] = i  # First occurrence index

    # Find common songs in all lists
    num_lists = len(lists)
    common_songs = [song for song, count in song_counts.items() if count == num_lists]

    # Sort based on first appearance
    common_songs.sort(key=lambda song: song_positions[song])

    return common_songs


In [ ]:
#Recommendating Songs based Playlist
li = ['Levitating', 'Psycho (feat. Ty Dolla $ign)', 'Blank Space', 'You Belong with Me', 'Look What You Made Me Do']

# Get recommendations for each song in the playlist
recommend = [recommend_song_annoy(song, df, X_scaled, annoy_index, top_n=10000) for song in li]

# Find common songs across all recommended lists
t = common_songs_priority(*recommend)

print(t)

['Money', 'Dreams', 'Why', 'Moonlight', 'Nobody', 'Dream', 'Home', 'Love', 'Without You', 'With You', 'Summertime', 'Trouble', 'One Day', 'Sunshine', 'Tonight', 'On My Own', 'Ghost', 'Forever', 'Alone', 'My Love', 'Temptation', 'Fever', 'Human', 'Drive']
